In [ ]:
!python -m spacy download en_core_web_sm

In [14]:
import pandas as pd
misinfo_transcripts = pd.read_csv('misinfo_trans_with_punctuation_cleaned.csv')
info_transcripts = pd.read_csv('info_trans_with_punctuation_cleaned.csv')

In [15]:
import spacy
nlp = spacy.load("en_core_web_trf")

def preprocess_text(text):
    # Apply spaCy pipeline
    doc = nlp(text)
    # Tokenize, remove stop words, and lemmatize
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

In [16]:
processed_misinfo_trans = misinfo_transcripts['transcript_with_punctuation'].apply(preprocess_text)
processed_info_trans = info_transcripts['transcript_with_punctuation'].apply(preprocess_text)

In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer(preprocessor=preprocess_text)
#dtm = vectorizer.fit_transform(processed_trans)
#dtm

In [ ]:
#dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
#dtm_df

In [17]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

In [18]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_misinfo_trans = lemmatization(processed_misinfo_trans)
lemmatized_info_trans = lemmatization(processed_info_trans)
#print (lemmatized_texts[0][0:90])

In [19]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

misinfo_data_words = gen_words(lemmatized_misinfo_trans)
info_data_words = gen_words(lemmatized_info_trans)

In [20]:
#Bigrams and trigrams
import gensim.models

misinfo_bigram_phrases = gensim.models.Phrases(misinfo_data_words, min_count=5, threshold=100)
misinfo_trigram_phrases = gensim.models.Phrases(misinfo_bigram_phrases[misinfo_data_words], threshold=100)

misinfo_bigram = gensim.models.phrases.Phraser(misinfo_bigram_phrases)
misinfo_trigram = gensim.models.phrases.Phraser(misinfo_trigram_phrases)

def make_misinfo_bigrams(texts):
    return([misinfo_bigram[doc] for doc in texts])

def make_misinfo_trigrams(texts):
    return ([misinfo_trigram[misinfo_bigram[doc]] for doc in texts])

misinfo_data_bigrams = make_misinfo_bigrams(misinfo_data_words)
misinfo_data_bigrams_trigrams = make_misinfo_trigrams(misinfo_data_bigrams)



info_bigram_phrases = gensim.models.Phrases(info_data_words, min_count=5, threshold=100)
info_trigram_phrases = gensim.models.Phrases(info_bigram_phrases[info_data_words], threshold=100)

info_bigram = gensim.models.phrases.Phraser(info_bigram_phrases)
info_trigram = gensim.models.phrases.Phraser(info_trigram_phrases)

def make_info_bigrams(texts):
    return([info_bigram[doc] for doc in texts])

def make_info_trigrams(texts):
    return ([info_trigram[info_bigram[doc]] for doc in texts])

info_data_bigrams = make_info_bigrams(info_data_words)
info_data_bigrams_trigrams = make_info_trigrams(info_data_bigrams)


In [21]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

misinfo_id2word = corpora.Dictionary(misinfo_data_bigrams_trigrams)

misinfo_texts = misinfo_data_bigrams_trigrams

misinfo_corpus = [misinfo_id2word.doc2bow(text) for text in misinfo_texts]
# print (corpus[0][0:20])

misinfo_tfidf = TfidfModel(misinfo_corpus, id2word=misinfo_id2word)

misinfo_low_value = 0.03
misinfo_words  = []
misinfo_words_missing_in_tfidf = []
for i in range(0, len(misinfo_corpus)):
    misinfo_bow = misinfo_corpus[i]
    misinfo_low_value_words = [] #reinitialize to be safe. You can skip this.
    misinfo_tfidf_ids = [id for id, value in misinfo_tfidf[misinfo_bow]]
    misinfo_bow_ids = [id for id, value in misinfo_bow]
    misinfo_low_value_words = [id for id, value in misinfo_tfidf[misinfo_bow] if value < misinfo_low_value]
    misinfo_drops = misinfo_low_value_words+misinfo_words_missing_in_tfidf
    for item in misinfo_drops:
        misinfo_words.append(misinfo_id2word[item])
    misinfo_words_missing_in_tfidf = [id for id in misinfo_bow_ids if id not in misinfo_tfidf_ids] # The words with tf-idf socre 0 will be missing

    misinfo_new_bow = [b for b in misinfo_bow if b[0] not in misinfo_low_value_words and b[0] not in misinfo_words_missing_in_tfidf]
    misinfo_corpus[i] = misinfo_new_bow



info_id2word = corpora.Dictionary(info_data_bigrams_trigrams)

info_texts = info_data_bigrams_trigrams

info_corpus = [info_id2word.doc2bow(text) for text in info_texts]
# print (corpus[0][0:20])

info_tfidf = TfidfModel(info_corpus, id2word=info_id2word)

info_low_value = 0.03
info_words  = []
info_words_missing_in_tfidf = []
for i in range(0, len(info_corpus)):
    info_bow = info_corpus[i]
    info_low_value_words = [] #reinitialize to be safe. You can skip this.
    info_tfidf_ids = [id for id, value in info_tfidf[info_bow]]
    info_bow_ids = [id for id, value in info_bow]
    info_low_value_words = [id for id, value in info_tfidf[info_bow] if value < info_low_value]
    info_drops = info_low_value_words+info_words_missing_in_tfidf
    for item in info_drops:
        info_words.append(info_id2word[item])
    info_words_missing_in_tfidf = [id for id in info_bow_ids if id not in info_tfidf_ids] # The words with tf-idf socre 0 will be missing

    info_new_bow = [b for b in info_bow if b[0] not in info_low_value_words and b[0] not in info_words_missing_in_tfidf]
    info_corpus[i] = info_new_bow


In [ ]:
#id2word = corpora.Dictionary(data_words)

#corpus = []
#for text in data_words:
#    new = id2word.doc2bow(text)
#    corpus.append(new)

#print (corpus[0][0:20])

#word = id2word[[0][:1][0]]
#print (word)

In [50]:
misinfo_lda_model = gensim.models.ldamodel.LdaModel(corpus=misinfo_corpus,
                                           id2word=misinfo_id2word,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [51]:
pyLDAvis.enable_notebook()
misinfo_vis = pyLDAvis.gensim.prepare(misinfo_lda_model, misinfo_corpus, misinfo_id2word, mds="mmds", R=30)
misinfo_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
20    -0.462201  0.296660       1        1  13.512758
16    -0.195218 -0.512736       2        1  13.358475
45    -0.429617 -0.329246       3        1   5.830115
40     0.530924 -0.029351       4        1   5.037826
31    -0.007555  0.523277       5        1   4.501046
42     0.449143 -0.241600       6        1   3.836427
19     0.287085 -0.435968       7        1   3.680144
46    -0.491434  0.063964       8        1   3.500207
6     -0.371440 -0.235333       9        1   2.610402
47     0.152243  0.466491      10        1   2.444319
12     0.284970  0.401947      11        1   2.065320
15    -0.305714  0.315090      12        1   2.037522
28    -0.214609  0.406974      13        1   1.985882
35     0.162946 -0.395265      14        1   1.964270
4      0.451235  0.141895      15        1   1.791373
13    -0.235202 -0.343171      16        1   1.695767
23     0.053084 -0.427845      17        1   1.631124
32    -0.432893 -0.101138      18        1   1.596953
36    -0.259417 -0.208391      19        1   1.572023
8     -0.394124  0.118828      20        1   1.527298
49     0.271148 -0.289468      21        1   1.479174
33    -0.057443 -0.439976      22        1   1.412431
9      0.343475 -0.171395      23        1   1.395260
41     0.063468 -0.324366      24        1   1.344971
34    -0.347360 -0.052290      25        1   1.264198
29    -0.065254  0.417356      26        1   1.226050
10     0.348079  0.260422      27        1   1.222889
24     0.212787 -0.197898      28        1   1.162407
7      0.362603  0.037248      29        1   1.160447
48     0.353675 -0.046829      30        1   1.130762
37    -0.116783 -0.308596      31        1   0.993033
0     -0.268024  0.180899      32        1   0.953531
17     0.172828  0.300759      33        1   0.939284
27    -0.215998  0.072574      34        1   0.919192
21    -0.233589 -0.027432      35        1   0.866970
3      0.214125  0.194594      36        1   0.864200
44    -0.005253  0.288468      37        1   0.852451
1     -0.141440  0.291627      38        1   0.828143
14     0.308206  0.139527      39        1   0.816589
22     0.084488  0.224183      40        1   0.758669
26    -0.093701  0.175886      41        1   0.575659
25     0.045382 -0.098795      42        1   0.496231
11    -0.054372 -0.167722      43        1   0.464284
5      0.172554 -0.018441      44        1   0.322029
2      0.016393  0.068439      45        1   0.194892
18    -0.002900 -0.017523      46        1   0.115523
38     0.015190  0.008415      47        1   0.015395
43     0.015177  0.008416      48        1   0.015393
39     0.015140  0.008418      49        1   0.015373
30     0.015192  0.008417      50        1   0.015318, topic_info=             Term        Freq        Total Category  logprob  loglift
179           eat  1022.00000  1022.000000  Default  30.0000  30.0000
126           oil  1093.00000  1093.000000  Default  29.0000  29.0000
19           food  1114.00000  1114.000000  Default  28.0000  28.0000
135         sugar   579.00000   579.000000  Default  27.0000  27.0000
148          know   727.00000   727.000000  Default  26.0000  26.0000
..            ...         ...          ...      ...      ...      ...
25   inflammation     0.00187   253.037389  Topic50  -8.6429  -3.0316
26         injury     0.00187     9.749379  Topic50  -8.6429   0.2247
27     joint_pain     0.00187    10.527184  Topic50  -8.6429   0.1480
28           kind     0.00187   181.206542  Topic50  -8.6429  -2.6977
29          large     0.00187    25.368367  Topic50  -8.6429  -0.7316

[2173 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
534      20  0.982402     ability
368      12  0.962342        able
0        42  0.823970    absolute
577      36  0.979052  absolutely
568      31  0.980954      absorb
...     ...       ...         ...

In [52]:
info_lda_model = gensim.models.ldamodel.LdaModel(corpus=info_corpus,
                                           id2word=info_id2word,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [53]:
pyLDAvis.enable_notebook()
info_vis = pyLDAvis.gensim.prepare(info_lda_model, info_corpus, info_id2word, mds="mmds", R=30)
info_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21     0.178740  0.483306       1        1  11.017434
18     0.324429  0.378774       2        1  10.749267
8     -0.275608  0.466900       3        1  10.418919
33    -0.049806  0.430129       4        1   6.929002
10     0.533970 -0.054269       5        1   5.154471
39     0.459483  0.185529       6        1   4.891596
14     0.101785  0.433101       7        1   4.824729
45    -0.461454 -0.158816       8        1   3.933083
31    -0.374575  0.277649       9        1   3.659708
36     0.224560  0.312114      10        1   3.577436
44     0.347557  0.210057      11        1   3.269865
43    -0.106548  0.511008      12        1   3.229645
6     -0.465233  0.141911      13        1   3.118145
47    -0.217820  0.243817      14        1   1.972524
26     0.240399  0.154453      15        1   1.820214
29    -0.306390  0.168387      16        1   1.698199
37    -0.117105  0.307195      17        1   1.468000
9     -0.291174 -0.007571      18        1   1.239361
46    -0.000389  0.163067      19        1   1.191011
34     0.340103  0.018391      20        1   1.138112
13    -0.196118  0.085017      21        1   1.110160
0      0.148548 -0.384095      22        1   1.078253
27     0.098253  0.185428      23        1   1.019354
48    -0.182561 -0.366154      24        1   0.907895
1     -0.427973 -0.054087      25        1   0.863488
3      0.371403 -0.084075      26        1   0.852548
12     0.281258 -0.226372      27        1   0.818909
5     -0.244502 -0.211203      28        1   0.784586
32     0.161295 -0.145409      29        1   0.738196
35    -0.025609 -0.407311      30        1   0.717166
40    -0.289652 -0.288218      31        1   0.644248
28     0.043324 -0.298172      32        1   0.623833
42    -0.255241 -0.106341      33        1   0.593087
7      0.280910 -0.342817      34        1   0.579709
23     0.130700 -0.242041      35        1   0.501709
17     0.263620 -0.130599      36        1   0.480881
16     0.114660 -0.023770      37        1   0.417805
4      0.029160 -0.200216      38        1   0.394218
19    -0.130076 -0.271842      39        1   0.366740
25    -0.050085 -0.287762      40        1   0.354529
11    -0.104598 -0.105152      41        1   0.301706
2     -0.050369 -0.029307      42        1   0.234922
38     0.015733 -0.102298      43        1   0.079727
24    -0.019231 -0.104872      44        1   0.078492
30    -0.001947 -0.078416      45        1   0.045302
41    -0.015516 -0.084108      46        1   0.042859
15    -0.007579 -0.090233      47        1   0.017377
20    -0.007583 -0.090239      48        1   0.017286
49    -0.007581 -0.090239      49        1   0.017197
22    -0.007570 -0.090231      50        1   0.017102, topic_info=        Term         Freq        Total Category  logprob  loglift
52       add  1067.000000  1067.000000  Default  30.0000  30.0000
16       eat   662.000000   662.000000  Default  29.0000  29.0000
22        go  1006.000000  1006.000000  Default  28.0000  28.0000
21       get   441.000000   441.000000  Default  27.0000  27.0000
69      food   501.000000   501.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
25  goodness     0.001623    18.847532  Topic50  -8.2789  -0.6858
26     green     0.001623    27.679613  Topic50  -8.2789  -1.0702
27      hear     0.001623    54.153743  Topic50  -8.2789  -1.7413
28  honestly     0.001623    18.738925  Topic50  -8.2789  -0.6801
29      kind     0.001623   119.638707  Topic50  -8.2789  -2.5339

[2289 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
586       4  0.355909        able
586      11  0.622841        able
1246      5  0.325181  absolutely
1246     18  0.569066  absolutely
757      10  0.884052      absorb
...     ...       ...         ...
298      10  0.526484        year
298      11  